In [2]:
import os
import sys
import time
import random
import re
import operator
from dotenv import load_dotenv, find_dotenv
from typing import List, Annotated
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

# --- LIBRARIES ---
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, get_buffer_string
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.document_loaders import WikipediaLoader
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Send 

# 1. Load Environment Variables
found_dotenv = find_dotenv()
if found_dotenv:
    load_dotenv(found_dotenv)
    print(f"✅ Loaded .env from: {found_dotenv}")
else:
    print("❌ Warning: .env file not found.")

if not os.getenv("GROQ_API_KEY") or not os.getenv("TAVILY_API_KEY"):
    raise ValueError("API Keys are missing. Check your .env file.")

✅ Loaded .env from: d:\Research Assistant Bot\Research-Assistant-Bot\.env


In [3]:
# 2. Initialize Models (WITH RATE LIMIT PROTECTION)

# The Planner: Smart, Structured (Llama 3.3 70B)
llm_planner = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

# The Worker: Fast, High Rate Limit (Llama 4 17B)
# FIX 1: Add .with_retry() to automatically handle 429 errors
llm_worker = ChatGroq(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
).with_retry(
    stop_after_attempt=8,         # Retry up to 8 times
    wait_exponential_jitter=True  # Randomly increase wait time (2s, 4s, 8s...)
)

print("✅ Models initialized with Auto-Retry protection.")

✅ Models initialized with Auto-Retry protection.


In [4]:
# --- DATA MODELS ---
class Analyst(BaseModel):
    affiliation: str = Field(description="Primary affiliation of the analyst.")
    name: str = Field(description="Name of the analyst.")
    role: str = Field(description="Role of the analyst in the context of the topic.")
    description: str = Field(description="Description of the analyst focus, concerns, and motives.")
    
    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"

class Perspectives(BaseModel):
    analysts: List[Analyst] = Field(description="Comprehensive list of analysts.")

class GenerateAnalystsState(TypedDict):
    topic: str
    max_analysts: int
    human_analyst_feedback: str
    analysts: List[Analyst]

class InterviewState(MessagesState):
    max_num_turns: int
    context: Annotated[list, operator.add]
    analyst: Analyst
    interview: str
    sections: list

class ResearchGraphState(TypedDict):
    topic: str 
    max_analysts: int 
    human_analyst_feedback: str 
    analysts: List[Analyst] 
    sections: Annotated[list, operator.add] 
    introduction: str 
    content: str 
    conclusion: str 
    final_report: str

In [5]:
# --- TOOLS & PROMPTS ---
tavily_search = TavilySearchResults(max_results=3) # Reduced to 2 to save tokens

analyst_instructions = """You are tasked with creating a set of AI analyst personas. 
Review the topic: {topic}
Review feedback: {human_analyst_feedback}
Pick the top {max_analysts} themes and assign one analyst to each."""

question_instructions = """You are an analyst interviewing an expert. 
Your goal is to gather interesting and specific insights.
Topic & Goals: {goals}
Introduce yourself, ask questions, and say 'Thank you so much for your help!' when done."""

answer_instructions = """You are an expert answering an analyst.
Use ONLY this context: {context}
Cite sources like [1] next to statements. List sources at the bottom."""

section_writer_instructions = """You are an expert technical writer. 
Your task is to create a section of a report based *strictly* on the provided source documents.

Target Audience: C-Level Executives and Technical Leads.
Tone: Professional, data-driven, and concise.

Instructions:
1. **Analyze:** Read the provided context. Identify key statistics, dates, quotes, and technical specifications.
2. **Draft:** Write a summary of the provided context.
   - Use strictly factual language.
   - If there are conflicting facts in the sources, mention the conflict.
   - Do NOT use phrases like "The text says" or "According to the document." Just state the facts.
3. **Structure:**
   - Start with a strong opening sentence summarizing the main insight.
   - Use bullet points for lists or features.
   - Limit length to ~300 words.

Title: {focus}"""

report_writer_instructions = """You are a Lead Research Editor compiling a final report on: {topic}

You have received memos from a team of analysts. Each memo contains specific insights and a list of "Raw Sources".

**Your Goal:**
Write a cohesive, professional "State of the Union" style report. Do NOT just copy-paste the memos one by one. Instead, synthesize the information into a unified narrative.

**Strict Requirements:**
1. **Thematic Organization:** Group related insights from different analysts together. If Analyst A and Analyst B both mentioned "Cost," combine those insights into a single "Financial Implications" section.
2. **Conflict Resolution:** If analysts provide conflicting data, present the range (e.g., "Estimates range from X to Y").
3. **Citation Handling:** - You MUST use the "Raw Sources" provided.
   - Cite statements using [1], [2], etc.
   - Ensure every claim is backed by a citation number.

**Output Structure:**
# {topic}

## Executive Summary
(A 3-sentence high-level overview of the findings)

## Key Insights
(The main body. Use subheaders like 'Market Trends', 'Technical Architecture', 'Risks', etc., based on the content. DO NOT use analyst names as headers.)

## Sources
(List the unique URLs provided in the memos, numbered [1], [2], etc.)

Memos to process: 
{context}"""


intro_conclusion_instructions = """Write a crisp {topic} Introduction or Conclusion.
Use headers: ## Introduction or ## Conclusion."""

C:\Users\prash\AppData\Local\Temp\ipykernel_22920\276714087.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavily_search = TavilySearchResults(max_results=3) # Reduced to 2 to save tokens


In [6]:
# --- NODES (PLANNER 70B) ---
def create_analysts(state: GenerateAnalystsState):
    topic = state['topic']
    max_analysts = state['max_analysts']
    feedback = state.get('human_analyst_feedback', '')
    structured_llm = llm_planner.with_structured_output(Perspectives)
    system_msg = analyst_instructions.format(topic=topic, human_analyst_feedback=feedback, max_analysts=max_analysts)
    analysts = structured_llm.invoke([SystemMessage(content=system_msg)]+[HumanMessage(content="Generate analysts.")])
    return {"analysts": analysts.analysts}

def write_report(state: ResearchGraphState):
    sections = state["sections"]
    topic = state["topic"]
    formatted_sections = "\n\n".join([f"{s}" for s in sections])
    system_msg = report_writer_instructions.format(topic=topic, context=formatted_sections)
    report = llm_planner.invoke([SystemMessage(content=system_msg)]+[HumanMessage(content="Write report.")])
    return {"content": report.content}

def write_introduction(state: ResearchGraphState):
    sections = state["sections"]
    topic = state["topic"]
    formatted_sections = "\n\n".join([f"{s}" for s in sections])
    instructions = intro_conclusion_instructions.format(topic=topic, formatted_str_sections=formatted_sections)
    intro = llm_planner.invoke([instructions]+[HumanMessage(content="Write introduction.")])
    return {"introduction": intro.content}

def write_conclusion(state: ResearchGraphState):
    sections = state["sections"]
    topic = state["topic"]
    formatted_sections = "\n\n".join([f"{s}" for s in sections])
    instructions = intro_conclusion_instructions.format(topic=topic, formatted_str_sections=formatted_sections)
    conclusion = llm_planner.invoke([instructions]+[HumanMessage(content="Write conclusion.")])
    return {"conclusion": conclusion.content}

# --- NODES (WORKER 17B) ---
def generate_question(state: InterviewState):
    # FIX 2: Small random sleep to prevent "thundering herd"
    time.sleep(random.uniform(2, 4))
    
    analyst = state["analyst"]
    messages = state["messages"]
    system_msg = question_instructions.format(goals=analyst.persona)
    question = llm_worker.invoke([SystemMessage(content=system_msg)]+messages)
    return {"messages": [question]}

def search_web(state: InterviewState):
    """ Search using Direct Prompting (More stable for 17B) """
    messages = state['messages']
    prompt = SystemMessage(content="Generate a concise web search query to help answer the analyst's last question. Return ONLY the query string.")
    response = llm_worker.invoke(messages + [prompt])
    search_query = response.content.strip('"').strip()
    
    try:
        results = tavily_search.invoke({"query": search_query})
        data = results if isinstance(results, list) else [results]
        formatted = "\n\n---\n\n".join(
            [f'<Document href="{doc.get("url", "")}"/>\n{doc.get("content", "")}\n</Document>' for doc in data]
        )
    except:
        formatted = f"Search failed for: {search_query}"

    return {"context": [formatted]}

def search_wikipedia(state: InterviewState):
    messages = state['messages']
    prompt = SystemMessage(content="Generate a concise Wikipedia search term. Return ONLY the term.")
    response = llm_worker.invoke(messages + [prompt])
    search_query = response.content.strip('"').strip()
    
    try:
        docs = WikipediaLoader(query=search_query, load_max_docs=3).load()
        formatted = "\n\n---\n\n".join(
            [f'<Document source="{d.metadata.get("source", "Wiki")}"/>\n{d.page_content}\n</Document>' for d in docs]
        )
    except:
        formatted = "No wikipedia results."
    return {"context": [formatted]}

def generate_answer(state: InterviewState):
    analyst = state["analyst"]
    messages = state["messages"]
    context = state["context"]
    system_msg = answer_instructions.format(goals=analyst.persona, context=context)
    answer = llm_worker.invoke([SystemMessage(content=system_msg)]+messages)
    answer.name = "expert"
    return {"messages": [answer]}

def save_interview(state: InterviewState):
    return {"interview": get_buffer_string(state["messages"])}

def write_section(state: InterviewState):
    # FIX 3: Longer random sleep for the heavy "Summary" node
    delay = random.uniform(5, 10)
    print(f"    (Summarizing... Staggering {delay:.1f}s)")
    time.sleep(delay)

    analyst = state["analyst"]
    context = state["context"]
    
    # Generate Summary
    system_msg = section_writer_instructions.format(focus=analyst.description)
    section = llm_worker.invoke([SystemMessage(content=system_msg)]+[HumanMessage(content=f"Use this source: {context}")])
    
    # FIX 4: Programmatically Extract URLs for High Quality Sources
    urls = re.findall(r'href="(.*?)"', str(context))
    section_content = section.content
    if urls:
        section_content += "\n\n### Raw Sources\n"
        for url in set(urls):
            section_content += f"- {url}\n"
            
    return {"sections": [section_content]}

In [7]:
# --- ROUTING ---
def human_feedback(state): pass

def should_continue(state):
    if state.get('human_analyst_feedback'): return "create_analysts"
    return "conduct_interview"

def route_messages(state, name="expert"):
    messages = state["messages"]
    if len([m for m in messages if isinstance(m, AIMessage) and m.name == name]) >= state.get('max_num_turns', 2):
        return 'save_interview'
    if "Thank you so much for your help" in messages[-2].content:
        return 'save_interview'
    return "ask_question"

def initiate_all_interviews(state):
    if state.get('human_analyst_feedback'): return "create_analysts"
    topic = state["topic"]
    return [Send("conduct_interview", {
        "analyst": analyst,
        "messages": [HumanMessage(content=f"So you said you were writing an article on {topic}?")]
    }) for analyst in state["analysts"]]

def finalize_report(state):
    content = state["content"].replace("## Insights", "").strip()
    if "## Sources" in content:
        content, sources = content.split("\n## Sources\n")
    else:
        sources = None
    final = f"{state['introduction']}\n\n---\n\n## Insights\n{content}\n\n---\n\n{state['conclusion']}"
    if sources: final += f"\n\n## Sources\n{sources}"
    return {"final_report": final}

In [8]:
# --- BUILD GRAPHS ---
interview_builder = StateGraph(InterviewState)
interview_builder.add_node("ask_question", generate_question)
interview_builder.add_node("search_web", search_web)
interview_builder.add_node("search_wikipedia", search_wikipedia)
interview_builder.add_node("answer_question", generate_answer)
interview_builder.add_node("save_interview", save_interview)
interview_builder.add_node("write_section", write_section)

interview_builder.add_edge(START, "ask_question")
interview_builder.add_edge("ask_question", "search_web")
interview_builder.add_edge("ask_question", "search_wikipedia")
interview_builder.add_edge("search_web", "answer_question")
interview_builder.add_edge("search_wikipedia", "answer_question")
interview_builder.add_conditional_edges("answer_question", route_messages, ['ask_question', 'save_interview'])
interview_builder.add_edge("save_interview", "write_section")
interview_builder.add_edge("write_section", END)
interview_graph = interview_builder.compile()

builder = StateGraph(ResearchGraphState)
builder.add_node("create_analysts", create_analysts)
builder.add_node("human_feedback", human_feedback)
builder.add_node("conduct_interview", interview_graph)
builder.add_node("write_report", write_report)
builder.add_node("write_introduction", write_introduction)
builder.add_node("write_conclusion", write_conclusion)
builder.add_node("finalize_report", finalize_report)

builder.add_edge(START, "create_analysts")
builder.add_edge("create_analysts", "human_feedback")
builder.add_conditional_edges("human_feedback", initiate_all_interviews, ["create_analysts", "conduct_interview"])
builder.add_edge("conduct_interview", "write_report")
builder.add_edge("conduct_interview", "write_introduction")
builder.add_edge("conduct_interview", "write_conclusion")
builder.add_edge(["write_conclusion", "write_report", "write_introduction"], "finalize_report")
builder.add_edge("finalize_report", END)

memory = MemorySaver()
graph = builder.compile(interrupt_before=['human_feedback'], checkpointer=memory)

print("🚀 Graph compiled successfully!")

🚀 Graph compiled successfully!


In [9]:
from IPython.display import Markdown

# CONFIGURATION
max_analysts = 3 
topic = "The Future of AI Agents in 2025"
thread = {"configurable": {"thread_id": "999"}} # Changed ID to clear history

# PHASE 1
print(f"🕵️ Generating Analysts...")
for event in graph.stream({"topic": topic, "max_analysts": max_analysts}, thread, stream_mode="values"):
    analysts = event.get('analysts', '')
    if analysts:
        print(f"✅ Generated {len(analysts)} Analysts.")
        for a in analysts:
            print(f"   - {a.name} ({a.role})")

# PHASE 2
print("\n🚀 Starting Parallel Research (This may take 2-3 minutes due to staggering)...")
graph.update_state(thread, {"human_analyst_feedback": None}, as_node="human_feedback")

for event in graph.stream(None, thread, stream_mode="updates"):
    node_name = next(iter(event.keys()))
    print(f"-- Finished Node: {node_name}")

final_state = graph.get_state(thread)
report = final_state.values.get('final_report')

print("\n\n" + "="*30)
print("FINAL REPORT")
print("="*30)
Markdown(report)

🕵️ Generating Analysts...
✅ Generated 3 Analysts.
   - Dr. Rachel Kim (AI Ethics Specialist)
   - Jack Taylor (Market Analyst)
   - Professor Liam Chen (Security Expert)

🚀 Starting Parallel Research (This may take 2-3 minutes due to staggering)...
    (Summarizing... Staggering 9.0s)
    (Summarizing... Staggering 8.5s)
    (Summarizing... Staggering 5.1s)
-- Finished Node: conduct_interview
-- Finished Node: conduct_interview
-- Finished Node: conduct_interview
-- Finished Node: write_introduction
-- Finished Node: write_conclusion
-- Finished Node: write_report
-- Finished Node: finalize_report


FINAL REPORT


## Introduction
As we approach 2025, the world is on the cusp of a revolution in artificial intelligence (AI). AI agents, once considered the stuff of science fiction, are now becoming an integral part of our daily lives. From virtual assistants to autonomous vehicles, AI agents are transforming the way we live, work, and interact with one another. With advancements in machine learning, natural language processing, and computer vision, AI agents are becoming increasingly sophisticated, enabling them to perform complex tasks with unprecedented accuracy and efficiency. As we look to the future, it is clear that AI agents will play a vital role in shaping the world of 2025 and beyond, bringing about unprecedented opportunities for innovation, growth, and transformation.

---

## Insights
# The Future of AI Agents in 2025

## Executive Summary
The year 2025 is poised to be a pivotal year for the adoption of AI agents, with significant implications for various industries and aspects of society. As AI agents become increasingly autonomous, capable of controlling other agents, buying goods and services, negotiating with one another, and creating new agents, concerns about bias, job displacement, and the need for new cultural norms are growing [1]. The market potential of AI agents is substantial, driven by their ability to operate autonomously in complex environments, prioritize decision-making over content creation, and integrate with various software tools [2]. However, the increasing use of AI agents also introduces new security risks, with researchers identifying a wave of attacks targeting AI agents' features [3].

## Key Insights

### Market Trends
The global AI agent market is projected to grow at a compound annual growth rate (CAGR) of over 35% between 2025 and 2030, reaching a valuation of approximately $45 billion by 2030 [4]. Estimates suggest that by 2026, 40% of enterprise apps will feature task-specific AI agents, up from less than 5% in 2025 [5]. The emergence of "vibe coding," a trend of using generative AI to spin up code from plain-language prompts, has democratized development but introduced security and reliability concerns [6].

### Technical Architecture
AI agents are becoming increasingly sophisticated, with enhanced reasoning capabilities, multi-modal integration, open-source development, local processing, and cost efficiency [7]. Major technology companies, such as Microsoft, Google, and Amazon Web Services, have offered platforms for deploying pre-built AI agents [8]. Notable AI agents and platforms include OpenAI Operator, ChatGPT Agent, Devin AI, and Perplexity AI [9].

### Financial Implications
The cost of implementing and maintaining AI agents is a significant consideration for organizations. While estimates vary, the cost of developing and deploying AI agents is expected to decrease as the technology becomes more widespread [10]. However, the potential cost savings from increased efficiency and productivity are substantial, with some estimates suggesting that AI agents could save organizations millions of dollars in labor costs [11].

### Risks
The growing use of AI agents introduces new security risks, with researchers identifying a wave of attacks targeting AI agents' features [12]. To mitigate these risks, organizations must design systems that can detect, contain, and recover from misuse [13]. This includes implementing controls to limit how far agent-driven attacks can spread and how much damage they can cause [14].

### Real-World Applications
AI agents are being applied in various industries, such as e-commerce, global supply chain, and healthcare [15]. In e-commerce, AI agents are handling customer inquiries, managing inventory, and optimizing pricing [16]. In the global supply chain, AI agents are coordinating logistics, sourcing materials, and managing distribution [17]. In healthcare, AI agents are providing 24/7 support for coverage queries, eligibility questions, or claim statuses [18].


---

## Conclusion
The future of AI agents in 2025 is poised to revolutionize numerous aspects of our lives, from healthcare and education to finance and transportation. As AI technology continues to advance at an unprecedented rate, we can expect to see more sophisticated and autonomous AI agents that can learn, adapt, and interact with humans in a more seamless and intuitive way. With the potential to drive significant economic growth, improve productivity, and enhance overall quality of life, the integration of AI agents into our daily lives is an exciting and inevitable development that will shape the world of tomorrow. As we embark on this new frontier, it is essential to prioritize responsible AI development, ensuring that these powerful technologies are harnessed for the betterment of society and humanity as a whole.

## Sources
1. https://www.salesforce.com/news/stories/future-of-ai-agents-2025/
2. https://thenewstack.io/ai-engineering-trends-in-2025-agents-mcp-and-vibe-coding/
3. https://unit42.paloaltonetworks.com/agentic-ai-threats/
4. https://www.gartner.com/en/newsroom/press-releases/2025-08-26-gartner-predicts-40-percent-of-enterprise-apps-will-feature-task-specific-ai-agents-by-2026-up-from-less-than-5-percent-in-2025
5. https://www.trendmicro.com/vinfo/us/security/news/threat-landscape/trend-micro-state-of-ai-security-report-1h-2025
6. https://medium.com/aimonks/building-multi-agent-ai-systems-in-2025-the-no-code-revolution-democratizing-enterprise-ai-a0be590d5b10
7. https://cloud.google.com/transform/ai-impact-industries-2025
8. https://www.oneadvanced.com/resources/future-of-ai-agents/
9. https://theriseofthedigitalworkforce.cio.com/theciosguidetoagenticai/thefutureofaiagents/
10. https://www.salesforce.com/au/news/stories/the-future-of-ai-agents-top-predictions-trends-to-watch-in-2026/
11. https://www.ibm.com/think/insights/ai-agents-2025-expectations-vs-reality
12. https://www.esecurityplanet.com/artificial-intelligence/ai-agent-attacks-in-q4-2025-signal-new-risks-for-2026/
13. https://www.linkedin.com/pulse/future-ai-agents-transforming-business-society-2025-beyond-john-enoh-hjo6c